In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib inline
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Finding Chessboard Conners and Distortion Coefficients

In [ ]:
# Finding chessboard conners
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

# Help function 

In [ ]:
# undistort the camera images
def calibrate_camera(image, mtx, dist):
    undist=cv2.undistort(image, mtx, dist, None, mtx)
    return undist

# set RGB threshhold
def rgb_thresh(img, thresh=(170, 255)):
    image_r=img[:,:,0]
    binary_r = np.zeros_like(image_r)
    binary_r[(image_r > thresh[0]) & (image_r <= thresh[1])] = 1
    
    return binary_r

# set threshold for s channel
def hls_thresh(img, thresh=(170, 255)):
    hls=cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel= hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    
    return binary_output

# set threshold for sobelx
def sobel_thresh(img, thresh=(40, 150)):
    sobelx=cv2.Sobel(img, cv2.CV_64F, 1,0)
    abs_sobelx=np.absolute(sobelx)
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    return sxbinary

# apply sliding window to fit curve
def sliding_windows(warped, video_name, nwindows=9, margin=40, minpix=50):
    
    histogram=np.sum(warped[warped.shape[0]//2:, :], axis=0)
    out_img = np.dstack((warped, warped, warped))
    mid_point=np.int(histogram.shape[0]//2)
    left_base=np.argmax(histogram[:mid_point])
    right_base=np.argmax(histogram[mid_point:])+mid_point
    
    nonzero = warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    window_height=np.int(warped.shape[0]//nwindows)
    leftx_current=left_base
    rightx_current=right_base
    
    left_lane_inds=[]
    right_lane_inds=[]
    
    for window in range(nwindows):
        win_y_low=warped.shape[0]-(window+1)*window_height
        win_y_high=warped.shape[0]-window*window_height
        
        win_xleft_low=leftx_current-margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        if video_name=='project':
            win_xleft_high = leftx_current + 1.8*margin
        elif video_name=='challenge':
            win_xleft_high = leftx_current + 1*margin
        else:
            win_xleft_high = leftx_current + 1.8*margin
        
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass 
    
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty

# fitting curve based on previous frame
def search_around_poly(warped, left_fit, right_fit):
    # Choose the width of the margin around the previous polynomial to search
    margin = 100

    # Grab activated pixels
    nonzero = warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty


# fit the lane lines
def poly_fit_curve(warped, leftx, lefty, rightx, righty):
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
    y_eval = np.max(ploty)
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        
    return left_fitx, right_fitx, ploty

# check the quality of the titted curve for the project video
def lane_sanity_check(left_fitx, right_fitx, ploty):
    flag = True
    lane_distance_bot = right_fitx[-1] - left_fitx[-1]
    lane_distance_mid = right_fitx[320] - left_fitx[320]
    lane_distance_top = right_fitx[0] - left_fitx[0]

    # tranform calibration distence 1280/2 is 640, 5%(610-670) is good search, 15%(545-730) is detected
    if ((lane_distance_bot < 545) or (lane_distance_bot > 730)): flag = False
    if ((lane_distance_mid < 545) or (lane_distance_mid > 730)): flag = False
    if ((lane_distance_top < 500) or (lane_distance_top > 730)): flag = False # change top to 500, in some frame, the road in not flat, the lane will be small far from camera

    return flag, lane_distance_bot, lane_distance_mid, lane_distance_top

# check the quality of the titted curve for the challenge video
def lane_sanity_check_challenge(left_fitx, right_fitx, ploty):

    flag = True
    lane_distance_bot = right_fitx[-1] - left_fitx[-1]
    lane_distance_mid = right_fitx[320] - left_fitx[320]
    lane_distance_top = right_fitx[0] - left_fitx[0]

    # tranform calibration distence 1280/2 is 640, 5%(610-670) is good search, 15%(545-730) is detected
    if ((lane_distance_bot < 480) or (lane_distance_bot > 600)): flag = False
    if ((lane_distance_mid < 350) or (lane_distance_mid > 500)): flag = False
    if ((lane_distance_top < 150) or (lane_distance_top > 500)): flag = False

    return flag, lane_distance_bot, lane_distance_mid, lane_distance_top

# check the quality of the titted curve for the harder challenge video
def lane_sanity_check_harder(left_fitx, right_fitx, ploty):

    flag = True
    lane_distance_bot = right_fitx[-1] - left_fitx[-1]
    lane_distance_mid = right_fitx[320] - left_fitx[320]
    lane_distance_top = right_fitx[0] - left_fitx[0]

    # tranform calibration distence 1280/2 is 640, 5%(610-670) is good search, 15%(545-730) is detected
    if ((lane_distance_bot < 400) or (lane_distance_bot > 700)): flag = False
    if ((lane_distance_mid < 400) or (lane_distance_mid > 1200)): flag = False
    if ((lane_distance_top < 200) or (lane_distance_top > 2200)): flag = False

    return (flag, lane_distance_bot, lane_distance_mid, lane_distance_top)

#calculate the curvature of the lane line
def cal_curvature(left_fitx, right_fitx, ploty, xm_per_pix=3.7/700, ym_per_pix=30/720):
    y_eval = np.max(ploty)
    left_temp_x = left_fitx[::-1]  # Reverse to match top-to-bottom in y
    right_temp_x = right_fitx[::-1]  # Reverse to match top-to-bottom in y

    # Fit a second order polynomial to pixel positions in each fake lane line
    left_fit = np.polyfit(ploty*ym_per_pix, left_temp_x*xm_per_pix, 2)
    right_fit = np.polyfit(ploty*ym_per_pix, right_temp_x*xm_per_pix, 2)
    left_curverad = ((1 + (2*left_fit[0]*y_eval*ym_per_pix + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval*ym_per_pix + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad

def find_curv(left_cur, right_cur):
    if np.abs(left_cur-1000)>np.abs(right_cur-1000):
        return right_cur
    else:
        return left_cur
    
# find offset value
def find_offset(img, warped, xm_per_pix=3.7/700):
    image_width = img.shape[1]
    img_mid=np.int(image_width//2)
    
    histogram=np.sum(warped[warped.shape[0]//2:, :], axis=0)
    mid_point=np.int(histogram.shape[0]//2)
    left_base=np.argmax(histogram[:mid_point])
    right_base=np.argmax(histogram[mid_point:])+mid_point
    real_mid=np.int((right_base-left_base)//2)+left_base
    offset=np.abs(real_mid-img_mid)*xm_per_pix
    
    return offset

# Create an image to draw the lines on
def draw_line(warped, img, left_fitx, right_fitx, ploty, Minv, image_size):
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, image_size) 
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    
    return result


# Pipeline to Process the Images
#### 1 Distortion correction
#### 2 Color & Gradient threshold
#### 3 Perspective transform
#### 4 Apply sliding windows to find the curve
#### 5 Measure curvature and offset distance

In [ ]:
# read image file names
images= glob.glob('test_images/*.jpg')
# pipeline to process the images to find the lane line
for fname in images:
    # load images and find image shape information
    image = mpimg.imread(fname)
    imshape = image.shape
    image_height = image.shape[0]
    image_width = image.shape[1]
    img_size = (image.shape[1], image.shape[0])
    
    # unditort grayscale images  
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    g_undist=calibrate_camera(gray, mtx, dist)
    # undistort color images
    undist=calibrate_camera(image, mtx, dist)
    
    # set rgb threshold
    binary_r=rgb_thresh(undist)
    # set hls threshold
    binary_hls=hls_thresh(undist)
    #set sobel threshold
    binary_sobelx=sobel_thresh(g_undist)
    
    #combine all the threshold results
    combined_binary = np.zeros_like(binary_sobelx)
    combined_binary[(binary_hls == 1) | (binary_sobelx == 1)] = 1
    
    # find src and dst points for perspective transform
    src = np.float32([[189, image_height], [572,464],[711,464], [1102,image_height]])
    dst = np.float32([[320,image_height], [320,0], [950,0], [950,image_height]])
    
    # perspective transform
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(combined_binary, M, img_size)
    
    # applying sliding window to fit the curve
    leftx, lefty, rightx, righty=sliding_windows(warped, video_name='project')
    left_fitx, right_fitx, ploty=poly_fit_curve(warped, leftx, lefty, rightx, righty)
    
    # calculate the curvature
    left_cur, right_cur=cal_curvature(left_fitx, right_fitx, ploty)
    #find curvature value
    curv_value=find_curv(left_cur, right_cur)
    #find offset distance
    offset=find_offset(image, warped)
    
    # Create an image to draw the lines on
    result=draw_line(warped, undist, left_fitx, right_fitx, ploty, Minv, img_size)
    texted_image =cv2.putText(img=np.copy(result), text="The curvature is: "+ str(np.round(curv_value,2))+" m", org=(150,100),fontFace=2, fontScale=2, color=(255,255,255), thickness=5)
    texted_image2 =cv2.putText(img=np.copy(texted_image), text="The offset is: "+ str(np.round(offset,4))+" m", org=(150,200),fontFace=2, fontScale=2, color=(255,255,255), thickness=5)
    
    cv2.imwrite('output_images/'+fname[12:],texted_image2)
    plt.imshow(texted_image2)
    plt.show()

# Pipeline for project video

In [ ]:
# Class to store the previous fitting information
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])] 
        #radius of curvature of the line in some units
        self.radius_of_curvature = [] 
        #distance in meters of vehicle center from the line
        self.line_base_pos = []
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = [] 
        #y values for detected line pixels
        self.ally = []

In [ ]:
# Pipeline for project video 
class Pipe_line():
    def __init__(self, left, right):
        self.left=left
        self.right=right
        self.search_ok=False
        self.search_fail=0
        self.fit_ok=False
        self.fit_fail_count=0
        self.avg_num=-3
        
    def get_current_search(self):
        leftx=self.left.allx[-1]
        lefty=self.left.ally[-1]
        rightx=self.right.allx[-1]
        righty=self.right.ally[-1]
        return leftx, lefty, rightx, righty
    
    def store_search(self, leftx, lefty, rightx, righty):
        self.left.allx.append(leftx)
        self.left.ally.append(lefty)
        self.right.allx.append(rightx)
        self.right.ally.append(righty)
        
    def get_current_fit(self):
        left_fitx=self.left.recent_xfitted[-1]
        right_fitx=self.right.recent_xfitted[-1]
        return left_fitx, right_fitx
    
    def get_avg_fit(self):
        left_fitx=np.average(self.left.recent_xfitted[self.avg_num:-1],axis=0)
        right_fitx=np.average(self.right.recent_xfitted[self.avg_num:-1], axis=0)
        return left_fitx, right_fitx
    
    def store_fit(self,left_fitx,right_fitx):
        self.left.recent_xfitted.append(left_fitx)
        self.right.recent_xfitted.append(right_fitx)
        
    def process_image(self, image):
        imshape = image.shape
        image_height = image.shape[0]
        image_width = image.shape[1]
        img_size = (image.shape[1], image.shape[0])

        # unditort grayscale images  
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        g_undist=calibrate_camera(gray, mtx, dist)
        # undistort color images
        undist=calibrate_camera(image, mtx, dist)

        # set rgb threshold
        binary_r=rgb_thresh(undist)
        # set hls threshold
        binary_hls=hls_thresh(undist)
        #set sobel threshold
        binary_sobelx=sobel_thresh(g_undist)

        #combine all the threshold results
        combined_binary = np.zeros_like(binary_sobelx)
        combined_binary[(binary_hls == 1) | (binary_sobelx == 1)] = 1

        # find src and dst points for perspective transform
        src = np.float32([[189, image_height], [572,464],[711,464], [1102,image_height]])
        dst = np.float32([[320,image_height], [320,0], [950,0], [950,image_height]])

        # perspective transform
        M = cv2.getPerspectiveTransform(src, dst)
        Minv = cv2.getPerspectiveTransform(dst, src)
        warped = cv2.warpPerspective(combined_binary, M, img_size)

        # applying sliding window to fit the curve
        
        leftx, lefty, rightx, righty=sliding_windows(warped, video_name='project')
        if leftx.size==0 or rightx.size==0:
            self.search_ok=False
            self.search_fail+=1
            if self.left.allx==[]:
                return image
            else:
                leftx, lefty, rightx, righty= self.get_current_search()
        else:
            self.search_ok=True
            self.store_search(leftx, lefty, rightx, righty)
        left_fitx, right_fitx, ploty=poly_fit_curve(warped, leftx, lefty, rightx, righty)
        lane_check_result = lane_sanity_check(left_fitx, right_fitx, ploty)
        self.fit_ok, lane_distance_bot, lane_distance_mid, lane_distance_top = lane_check_result
        
        if self.fit_ok:
            self.store_fit(left_fitx, right_fitx)
        elif self.left.recent_xfitted==[]:
            self.fit_fail_count+=1
        else:
            self.fit_fail_count += 1
            left_fitx, right_fitx=self.get_current_fit()
            leftx, lefty, rightx, righty = self.get_current_search()
                        
        # calculate the curvature
        left_cur, right_cur=cal_curvature(left_fitx, right_fitx, ploty)
        #find curvature value
        curv_value=find_curv(left_cur, right_cur)
        #find offset distance
        offset=find_offset(image, warped)

        # Create an image to draw the lines on
        result_temp=draw_line(warped, undist, left_fitx, right_fitx, ploty, Minv, img_size)
        texted_image =cv2.putText(img=np.copy(result_temp), text="The curvature is: "+ str(np.round(curv_value,2))+" m", org=(150,100),fontFace=2, fontScale=2, color=(255,255,255), thickness=5)
        result =cv2.putText(img=np.copy(texted_image), text="The offset is: "+ str(np.round(offset,4))+" m", org=(150,200),fontFace=2, fontScale=2, color=(255,255,255), thickness=5)

        return result
    

In [ ]:
left=Line()
right=Line()
p=Pipe_line(left, right)
project_video_output = 'output_images/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(p.process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(project_video_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_video_output))

# Pipeline for challenge video 

In [ ]:
# Pipeline for challenge video 
class Pipe_line():
    def __init__(self, left, right):
        self.left=left
        self.right=right
        self.search_ok=False
        self.search_fail=0
        self.fit_ok=False
        self.fit_fail_count=0
        self.avg_num=-15
        
    def get_current_search(self):
        leftx=self.left.allx[-1]
        lefty=self.left.ally[-1]
        rightx=self.right.allx[-1]
        righty=self.right.ally[-1]
        return leftx, lefty, rightx, righty
    
    def store_search(self, leftx, lefty, rightx, righty):
        self.left.allx.append(leftx)
        self.left.ally.append(lefty)
        self.right.allx.append(rightx)
        self.right.ally.append(righty)
        
    def get_current_fit(self):
        left_fitx=self.left.recent_xfitted[-1]
        right_fitx=self.right.recent_xfitted[-1]
        return left_fitx, right_fitx
    
    def get_avg_fit(self):
        left_fitx=np.average(self.left.recent_xfitted[self.avg_num:-1],axis=0)
        right_fitx=np.average(self.right.recent_xfitted[self.avg_num:-1], axis=0)
        return left_fitx, right_fitx
    
    def store_fit(self,left_fitx,right_fitx):
        self.left.recent_xfitted.append(left_fitx)
        self.right.recent_xfitted.append(right_fitx)
        
    
    def process_image_challenge(self, image):
        imshape = image.shape
        image_height = image.shape[0]
        image_width = image.shape[1]
        img_size = (image.shape[1], image.shape[0])

        # unditort grayscale images  
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        g_undist=calibrate_camera(gray, mtx, dist)
        # undistort color images
        undist=calibrate_camera(image, mtx, dist)

        # set rgb threshold
        binary_r=rgb_thresh(undist)
        # set hls threshold
        binary_hls=hls_thresh(undist, thresh=(100, 180))
        #set sobel threshold
        binary_sobelx=sobel_thresh(g_undist)

        #combine all the threshold results
        combined_binary = np.zeros_like(binary_sobelx)
        combined_binary[(binary_hls == 1) | (binary_sobelx == 1)] = 1

        # find src and dst points for perspective transform

        src = np.float32([[189, image_height], [520,490],[730,490], [1102,image_height]])
        dst = np.float32([[320,image_height], [320,0], [950,0], [950,image_height]])

        # perspective transform
        M = cv2.getPerspectiveTransform(src, dst)
        Minv = cv2.getPerspectiveTransform(dst, src)
        warped = cv2.warpPerspective(combined_binary, M, img_size)

        # applying sliding window to fit the curve
        
        leftx, lefty, rightx, righty=sliding_windows(warped, video_name='challenge')
        #leftx, lefty, rightx, righty=sliding_windows_challenge(warped)
        if leftx.size==0 or rightx.size==0:
            self.search_ok=False
            self.search_fail+=1
            if self.left.allx==[]:
                return image
            else:
                leftx, lefty, rightx, righty= self.get_current_search()
        else:
            self.search_ok=True
            self.store_search(leftx, lefty, rightx, righty)
        left_fitx, right_fitx, ploty=poly_fit_curve(warped, leftx, lefty, rightx, righty)
        lane_check_result = lane_sanity_check_challenge(left_fitx, right_fitx, ploty)
        self.fit_ok, lane_distance_bot, lane_distance_mid, lane_distance_top = lane_check_result
        
        if self.fit_ok:
            self.store_fit(left_fitx, right_fitx)
        elif self.left.recent_xfitted==[]:
            self.fit_fail_count+=1
        else:
            self.fit_fail_count += 1
            if self.fit_fail_count<3:
                left_fitx, right_fitx=self.get_current_fit()
                leftx, lefty, rightx, righty = self.get_current_search()
            else:
                left_fitx, right_fitx = self.get_avg_fit()
                leftx, lefty, rightx, righty = self.get_current_search()
            
        # calculate the curvature
        left_cur, right_cur=cal_curvature(left_fitx, right_fitx, ploty)
        #find curvature value
        curv_value=find_curv(left_cur, right_cur)
        #find offset distance
        offset=find_offset(image, warped)

        # Create an image to draw the lines on
        result_temp=draw_line(warped, undist, left_fitx, right_fitx, ploty, Minv, img_size)
        texted_image =cv2.putText(img=np.copy(result_temp), text="The curvature is: "+ str(np.round(curv_value,2))+" m", org=(150,100),fontFace=2, fontScale=2, color=(255,255,255), thickness=5)
        result =cv2.putText(img=np.copy(texted_image), text="The offset is: "+ str(np.round(offset,4))+" m", org=(150,200),fontFace=2, fontScale=2, color=(255,255,255), thickness=5)

        return result

In [ ]:
left=Line()
right=Line()
p=Pipe_line(left, right)
challenge_video_output = 'output_images/challenge_video.mp4'
#clip1 = VideoFileClip("challenge_video.mp4").subclip(0,5)
clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(p.process_image_challenge) #NOTE: this function expects color images!!
%time white_clip.write_videofile(challenge_video_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_video_output))

# Pipeline for harder challenge video

In [ ]:
# Pipeline for harder challenge video 
class Pipe_line():
    def __init__(self, left, right):
        self.left=left
        self.right=right
        self.search_ok=False
        self.search_fail=0
        self.fit_ok=False
        self.fit_fail_count=0
        self.avg_num=-10
        
    def get_current_search(self):
        leftx=self.left.allx[-1]
        lefty=self.left.ally[-1]
        rightx=self.right.allx[-1]
        righty=self.right.ally[-1]
        return leftx, lefty, rightx, righty
    
    def store_search(self, leftx, lefty, rightx, righty):
        self.left.allx.append(leftx)
        self.left.ally.append(lefty)
        self.right.allx.append(rightx)
        self.right.ally.append(righty)
        
    def get_current_fit(self):
        left_fitx=self.left.recent_xfitted[-1]
        right_fitx=self.right.recent_xfitted[-1]
        return left_fitx, right_fitx
    
    def get_avg_fit(self):
        left_fitx=np.average(self.left.recent_xfitted[self.avg_num:-1],axis=0)
        right_fitx=np.average(self.right.recent_xfitted[self.avg_num:-1], axis=0)
        return left_fitx, right_fitx
    
    def store_fit(self,left_fitx,right_fitx):
        self.left.recent_xfitted.append(left_fitx)
        self.right.recent_xfitted.append(right_fitx)
        
    def bal_brightness(self, input_img, value=80):
        hsv = cv2.cvtColor(input_img, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)

        lim = 255 - value
        v[v < lim] +=value
        v[v >= lim] -= value

        final_hsv = cv2.merge((h, s, v))
        out_img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
        return out_img
        
    def process_image_harder(self, image):
        imshape = image.shape
        image_height = image.shape[0]
        image_width = image.shape[1]
        img_size = (image.shape[1], image.shape[0])
        
        #Balance the brigntness of the image
        b_img=self.bal_brightness(input_img=np.copy(image), value=60)
        
        # unditort grayscale images  
        gray = cv2.cvtColor(b_img,cv2.COLOR_BGR2GRAY)
        g_undist=calibrate_camera(gray, mtx, dist)
        # undistort color images
        undist=calibrate_camera(b_img, mtx, dist)
        undist_original=calibrate_camera(image, mtx, dist)
        
        # set rgb threshold
        binary_r=rgb_thresh(undist)
        # set hls threshold
        binary_hls=hls_thresh(undist, thresh=(120, 180))
        #set sobel threshold
        binary_sobelx=sobel_thresh(g_undist)

        #combine all the threshold results
        combined_binary = np.zeros_like(binary_sobelx)
        combined_binary[(binary_hls == 1) | (binary_sobelx == 1)] = 1

        # find src and dst points for perspective transform
        #src = np.float32([[189, image_height], [400,600],[850,600], [1102,image_height]])
        src = np.float32([[189, image_height], [450,550],[840,550], [1102,image_height]])
        #src = np.float32([[189, image_height], [400,600],[850,600], [1102,image_height]])
        dst = np.float32([[320,image_height], [320,0], [950,0], [950,image_height]])

        # perspective transform
        M = cv2.getPerspectiveTransform(src, dst)
        Minv = cv2.getPerspectiveTransform(dst, src)
        warped = cv2.warpPerspective(combined_binary, M, img_size)

        # applying sliding window to fit the curve
        
        leftx, lefty, rightx, righty=sliding_windows(warped, video_name='challenge')
        if leftx.size==0 or rightx.size==0:
            self.search_ok=False
            self.search_fail+=1
            if self.left.allx==[]:
                return image
            else:
                leftx, lefty, rightx, righty= self.get_current_search()
        else:
            self.search_ok=True
            self.store_search(leftx, lefty, rightx, righty)
        left_fitx, right_fitx, ploty=poly_fit_curve(warped, leftx, lefty, rightx, righty)
        lane_check_result = lane_sanity_check_harder(left_fitx, right_fitx, ploty)
        self.fit_ok, lane_distance_bot, lane_distance_mid, lane_distance_top = lane_check_result
        
        if self.fit_ok:
            self.store_fit(left_fitx, right_fitx)
        elif self.left.recent_xfitted==[]:
            self.fit_fail_count+=1
        else:
            self.fit_fail_count += 1
            #left_fitx, right_fitx=self.get_current_fit()
            left_fitx, right_fitx = self.get_avg_fit()
            leftx, lefty, rightx, righty = self.get_current_search()
            
        # calculate the curvature
        left_cur, right_cur=cal_curvature(left_fitx, right_fitx, ploty)
        #find curvature value
        curv_value=find_curv(left_cur, right_cur)
        #find offset distance
        offset=find_offset(image, warped)

        # Create an image to draw the lines on
        result_temp=draw_line(warped, undist_original, left_fitx, right_fitx, ploty, Minv, img_size)
        texted_image =cv2.putText(img=np.copy(result_temp), text="The curvature is: "+ str(np.round(curv_value,2))+" m", org=(150,100),fontFace=2, fontScale=2, color=(255,255,255), thickness=5)
        result =cv2.putText(img=np.copy(texted_image), text="The offset is: "+ str(np.round(offset,4))+" m", org=(150,200),fontFace=2, fontScale=2, color=(255,255,255), thickness=5)

        return result

In [ ]:
left=Line()
right=Line()
p=Pipe_line(left, right)
harder_video_output = 'output_images/harder_challenge_video.mp4'
#clip1 = VideoFileClip("harder_challenge_video.mp4").subclip(0,15)
clip1 = VideoFileClip("harder_challenge_video.mp4")
white_clip = clip1.fl_image(p.process_image_harder) #NOTE: this function expects color images!!
%time white_clip.write_videofile(harder_video_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(harder_video_output))

In [ ]:
# save video into gif files
def gif_from_video(video_path_prefix, s=(0,0), f=(0,47)):
    video_file = 'output_images/{}.mp4'.format(video_path_prefix)
    
    clip = (VideoFileClip(video_file).subclip(s,f).resize(0.5))
    %time clip.write_gif('output_images/{}.gif'.format(video_path_prefix), fps=10)

In [ ]:
gif_from_video('project_video',s=(0, 0), f=(0,47))

In [ ]:
gif_from_video('challenge_video', s=(0,0), f=(0,16))

In [ ]:
gif_from_video('harder_challenge_video',s=(0, 0), f=(0,47))